# Train CNN

Vary the data used to train different models with noisy or coarse d

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import time
import xarray as xr
import pickle
import matplotlib.colors as colors

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
import sys

In [2]:
torch.manual_seed(0) # for reproduceability

In [3]:
def zc():
    return np.array([-481.68085,-446.76865,-414.0377,-383.35178,-354.58304,-327.6118,-302.32565,-278.61935,
                           -256.39423,-235.5577,-216.02301,-197.70883,-180.5389,-164.44174,-149.3503,-135.20177,
                           -121.937225,-109.50143,-97.84261,-86.91223,-76.664764,-67.05755,-58.050587,-49.60637,
                           -41.689735,-34.267727,-27.309437,-20.785896,-14.669938,-8.9361,-3.5605056,-0.5])



## Load data

In [7]:
class psom_data(Dataset):
    """creates dataset from model outputs for training based on list of simulations to use and
       range of days to include"""

    def __init__(self, res=1, datadir='data', noise = 0, days=(5,10), var = (0,1,3,5,6), transform=None, standardize=True, basic=True):
        """
        Args:
            datadir (string): path to model outputs
            day (int): which day of analysis period to use (start_day + day)
            var (tuple): indices of variables to use
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """     
        X = []; Y = []; center = []; exp = []; day_list = [];
        i1 = []; i2 = []; j1 = []; j2 = []
        
        for day in days:
            with open(os.path.join(datadir,'coarse_%ikm_day%i.pkl'%(res,day)),'rb') as f:
                X_temp, Y_temp, center_temp, coord_temp, exp_temp = pickle.load(f)
        
            X.extend(X_temp); Y.extend(Y_temp); center.extend(center_temp); 
            i1.extend(coord_temp[0]); i2.extend(coord_temp[1]);
            j1.extend(coord_temp[2]); j2.extend(coord_temp[3]);
            exp.extend(exp_temp); day_list.extend([day]*len(X_temp))
                
        if standardize:
            Xarr = np.array(X)
            mean = np.mean(Xarr, axis=(0,2,3), keepdims=True)
            std = np.std(Xarr, axis=(0,2,3), keepdims=True)
            X = (Xarr-mean)/std
            
        self.X = X # list of inputs, each sample has dim (N_var, n, n)
        self.Y = Y # list of outputs, each sample has dim (depth,)
        self.center = center
        self.i1 = i1
        self.i2 = i2
        self.j1 = j1
        self.j2 = j2
        self.exp = exp
        self.transform = transform
        self.basic=basic
        self.day = day_list
        self.var = var

    def __len__(self):
        assert len(self.X)==len(self.Y), 'X and Y must have the same lengths'
        assert len(self.exp)==len(self.i1), 'coordinates and experiments must have same lengths'
        assert len(self.i1)==len(self.i2)==len(self.j1)==len(self.j2), 'coordinates must have same lengths'
        return len(self.X)  

    def __getitem__(self, idx):
        if self.transform:
            sample = self.transform(sample)
            
        sample = {'X': self.X[idx][self.var,:,:], 'Y': self.Y[idx], 'center': self.center[idx],
          'coord1': (self.j1[idx], self.i1[idx]), 'coord2': (self.j2[idx], self.i2[idx]), 'exp': self.exp[idx]}
            
        if self.basic:
            return sample['X'],sample['Y']
        else:
            return sample['X'], sample['Y'], sample['center'], sample['coord1'], sample['coord2'], sample['exp']

## load data and split into training/validation/testing

In [8]:
def split_data(dataset, valid_size):
    """
    Randomly split the data into a training and validation set. Returns the a set of samples for training and validation.
    """    
    #dataset = psom_data(datadir=datadir, days=days, basic=True, standardize=True)

    data_idx = [i for i in range(len(dataset))]
        
    # shuffling the indices and adding them to the current training and testing sets
    np.random.shuffle(data_idx)
    split = int(np.floor(valid_size * len(data_idx)))
        
    train_idx = data_idx[split:]
    test_idx = data_idx[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    return train_sampler, test_sampler

def load_data(datadir = 'data', days = [5], res = 1, noise = 0, var=(0,1,3,5,6), valid_size = .2, batchsize=1):
    """
    Randomly split the data into a training and validation set. Returns the a set of samples for training and validation.
    """
    print('reading in data...')
    dataset = psom_data(datadir=datadir, res = res, noise = noise, days=days, var=var, basic=True, standardize=True)
    
    print('shuffling data...')
    train_sampler, test_sampler = split_data(dataset, valid_size)
    
    print('loading train and test loader...')
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batchsize, 
                                               sampler=train_sampler)
    
    test_loader = torch.utils.data.DataLoader(dataset, batch_size=batchsize, 
                                               sampler=test_sampler)
    return train_loader, test_loader

## Simple CNN

In [9]:
# model with 3 convolutional layers and 2 fully connected layers
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # layer 1 consists of conv, relu, MP
        self.conv1 = nn.Sequential(nn.Conv2d(5, 16, kernel_size=5, padding=0), nn.ReLU(inplace=True),
                                   nn.MaxPool2d(kernel_size=2, stride=2))
        
        # layer 2 consists of conv, relu, MP
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=3, padding=0), nn.ReLU(inplace=True),
                                   nn.MaxPool2d(kernel_size=2, stride=2))

        # layer 3 consists of conv, relu, MP
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, padding=0), nn.ReLU(inplace=True),
                                   nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc1 = nn.Sequential(nn.Linear(256, 64), nn.ReLU(inplace=True))
        self.fc2 = nn.Sequential(nn.Linear(64, 32)) # output is a profile of w


    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(-1, 256)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Train

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

print(device)

def train(model, train_loader,batchsize=1,lr=0.01):
    model.train()
    i=0
    avg_loss = 0
    for X,Y in train_loader:
        i+=1
        optimizer = optim.SGD(net.parameters(), lr=lr)
        optimizer.zero_grad()
        model.zero_grad()
        
        # forward pass
        tensor = X.view(X.shape[0],X.shape[1],X.shape[2],X.shape[3]).float()
        tensor = tensor.to(device)
        Y = Y.to(device)

        output = model(tensor)
        loss = F.mse_loss(output, Y.float())
        avg_loss += loss.item()
        
        # backward pass
        loss.backward()
        optimizer.step()
    return avg_loss/(len(train_loader))

def test(model, test_loader, batchsize=1):
    model.eval()
    
    loss = 0
    Lprofile = np.zeros(32)
    w_mag = np.zeros(32)
    
    for X,Y in test_loader:
        tensor = X.view(X.shape[0],X.shape[1],X.shape[2],X.shape[3]).float()
        tensor = tensor.to(device)
        Y = Y.to(device)
        pred = model(tensor)
        
        loss += F.mse_loss(pred, Y.float()).item()
        
        # calculate MSE at each depth
        Lprofile += np.sum((pred.cpu().detach().numpy()-Y.cpu().detach().numpy())**2, axis=0).flatten()
        
        # calculate magnitude of w at each depth
        w_mag += np.sum((Y.cpu().detach().numpy())**2, axis=0).flatten()
        
        #print(loss)
    mean_Lprofile = Lprofile/len(test_loader)
    rms = (mean_Lprofile)**0.5
    w_rms = (w_mag/len(test_loader))**0.5
    
    return loss/len(test_loader), rms, w_rms

cuda


In [13]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Train model, res = 10km, noise=0

In [15]:
trainloader, testloader_null = load_data(datadir = 'data/coarse/', res = 10, days = [0,10,15,20], var = (0,1,3,5,6), batchsize=1,valid_size = 0)
print(len(trainloader))

reading in data...
shuffling data...
loading train and test loader...
238032


In [24]:
# vary these--------
res = 15
batch_size=8
noise_level = 0
epochs = 20
learning_rate = 1e-3
model_dir = 'models/v3_coarse15km_1e-3'
#--------------------

# load data
s0 = time.time()
trainloader, testloader_null = load_data(datadir = 'data/coarse', res = res, noise = noise_level, days = [0,10,15,20], var = (0,1,3,5,6), batchsize=batch_size,valid_size = 0)
del testloader_null
trainloader_null, testloader = load_data(datadir = 'data/coarse', res = res, noise = noise_level, days = [5], var=(0,1,3,5,6), batchsize=batch_size,valid_size = 1)
del trainloader_null
print('time to load data: %0.2f min' %((time.time()-s0)/60))

# Train
net = Net()
net.to(device)

loss_epochs=[]
test_loss=[]

for i in range(epochs):   
    print('epoch %i'%i)
    
    # train
    s1 = time.time()
    loss = train(net, trainloader, batchsize=batch_size, lr=learning_rate)
    loss_epochs.append(loss)
    print('time to train epoch: %0.2f min' %((time.time()-s1)/60))
    
    print('mse train loss: %0.4f' %loss)
    tloss, rms, w_rms = test(net, testloader)
    test_loss.append(tloss)
    print('mse test loss: %0.4f' %tloss)
    
# Save

torch.save(net.state_dict(), model_dir+'/trained_cnn.pt')
np.save(model_dir+'/train_loss.npy',np.array(loss_epochs))
np.save(model_dir+'/test_loss.npy',np.array(test_loss))
np.save(model_dir+'/test_wrms_profile.npy',np.array(w_rms))
np.save(model_dir+'/test_rms_profile.npy',np.array(rms))

# Plot
plt.plot(loss_epochs, 'o-',label='train loss')
plt.plot(test_loss, 'o-', label='test loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('mse loss')

reading in data...
shuffling data...
loading train and test loader...
reading in data...
shuffling data...
loading train and test loader...
time to load data: 1.25 min
epoch 0
time to train epoch: 1.49 min
mse train loss: 0.0567
mse test loss: 0.0461
epoch 1
time to train epoch: 1.49 min
mse train loss: 0.0535
mse test loss: 0.0433
epoch 2
time to train epoch: 1.48 min
mse train loss: 0.0505
mse test loss: 0.0412
epoch 3
time to train epoch: 1.49 min
mse train loss: 0.0484
mse test loss: 0.0397
epoch 4
time to train epoch: 1.49 min
mse train loss: 0.0460
mse test loss: 0.0382
epoch 5
time to train epoch: 1.49 min
mse train loss: 0.0439
mse test loss: 0.0371
epoch 6
time to train epoch: 1.49 min
mse train loss: 0.0422
mse test loss: 0.0365
epoch 7
time to train epoch: 1.49 min
mse train loss: 0.0407
mse test loss: 0.0356
epoch 8
time to train epoch: 1.49 min
mse train loss: 0.0391
mse test loss: 0.0345
epoch 9
time to train epoch: 1.49 min
mse train loss: 0.0373
mse test loss: 0.0334
ep

FileNotFoundError: [Errno 2] No such file or directory: 'models/v3_coarse15km_1e-3/trained_cnn.pt'